# Colab Full-Run Notebook (ABP + PPG)

This notebook downloads the **full** datasets from Google Drive, builds subject-level indices, and trains the ResNet50 scalogram models on Colab GPU.

**Before you start:**
- Enable GPU: **Runtime → Change runtime type → GPU**.
- Upload/clone this project into Colab at `/content/intracranial-aneurysm` (or update `PROJECT_ROOT` below).

In [ ]:
import os
from pathlib import Path

PROJECT_ROOT = "/content/intracranial-aneurysm"  # update if you use a different folder
if not os.path.isdir(PROJECT_ROOT):
    raise FileNotFoundError(
        "PROJECT_ROOT not found. Upload/clone the repo into /content/intracranial-aneurysm or update PROJECT_ROOT."
    )

os.chdir(PROJECT_ROOT)
print("PROJECT_ROOT:", PROJECT_ROOT)

In [ ]:
!pip -q install -r requirements.txt
!pip -q install gdown

In [ ]:
from pathlib import Path
import subprocess

DATA_ROOT = Path(PROJECT_ROOT)
folders = {
    "ppg_NONCAD": "https://drive.google.com/drive/folders/1ogk72b6ppHUwJy-jPqu_HzZJ9cVgeAEV?usp=drive_link",
    "ppg_CAD": "https://drive.google.com/drive/folders/1Rtz7YFRQ2544B1LvC7RTO9wWxMp-bu6O?usp=drive_link",
    "ABPNONCAD": "https://drive.google.com/drive/folders/14ab13FVVjORknHKlccxTho_ksPw0zBZn?usp=drive_link",
    "ABPCAD": "https://drive.google.com/drive/folders/1mnQmZqsDN9K-ig4bu4GZdUTccNkm_95h?usp=drive_link",
}

for name, url in folders.items():
    out_dir = DATA_ROOT / name
    out_dir.mkdir(parents=True, exist_ok=True)
    if any(out_dir.glob("*.mat")):
        print(f"{name}: already has .mat files, skipping download")
        continue
    print(f"Downloading {name}...")
    subprocess.run(["gdown", "--folder", url, "-O", str(out_dir)], check=True)

print("Download complete.")

In [ ]:
import yaml
from pathlib import Path

cfg_paths = [Path("configs/abp.yaml"), Path("configs/ppg.yaml")]
for cfg_path in cfg_paths:
    cfg = yaml.safe_load(cfg_path.read_text())
    cfg["data_root"] = str(Path(PROJECT_ROOT).resolve())
    cfg["index_dir"] = str((Path(PROJECT_ROOT) / "data/index").resolve())
    cfg["output_dir"] = str((Path(PROJECT_ROOT) / "outputs").resolve())
    cfg["tiny_run"] = False  # full run on Colab/GPU
    cfg_path.write_text(yaml.safe_dump(cfg, sort_keys=False))
    print(f"Updated {cfg_path}")

In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

In [ ]:
!python scripts/make_index.py --config configs/abp.yaml
!python scripts/make_index.py --config configs/ppg.yaml

In [ ]:
!python scripts/train_resnet50.py --config configs/abp.yaml
!python scripts/train_resnet50.py --config configs/ppg.yaml

In [ ]:
# Optional: classical baselines
!python scripts/baseline_features.py --config configs/abp.yaml
!python scripts/baseline_features.py --config configs/ppg.yaml

In [ ]:
import json
from pathlib import Path

report_files = [
    Path("outputs/metrics_abp.json"),
    Path("outputs/metrics_ppg.json"),
    Path("outputs/baseline_abp.json"),
    Path("outputs/baseline_ppg.json"),
]

for p in report_files:
    if p.exists():
        print(f"\n{p}:")
        print(json.dumps(json.loads(p.read_text()), indent=2))
    else:
        print(f"Missing: {p}")

In [ ]:
# Generate methods/results report
!python scripts/generate_report.py --configs configs/abp.yaml configs/ppg.yaml --out docs/REPORT.md

print("Report saved to docs/REPORT.md")

In [ ]:
# Display figures inline
from pathlib import Path
from IPython.display import display
from PIL import Image

fig_dir = Path("outputs/figures")
if fig_dir.exists():
    for p in sorted(fig_dir.glob("*.png")):
        print(p)
        display(Image.open(p))
else:
    print("No figures found. Run training first.")